# Store item demand

In [ ]:
__author__ = "Zhiji Ding"
__copyright__ = "Copyright 2020, Zhiji Ding"
__email__ = "zhiji.ding@gmail.com"

In [1]:
# Module import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplot
import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# load data
train_df=pd.read_csv('/Users/jimmyding/Downloads/demand_forecasting_kernels_only/train.csv')
test_df=pd.read_csv('/Users/jimmyding/Downloads/demand_forecasting_kernels_only/test.csv')

In [3]:
train_df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
test_df.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [5]:
train_df['date']=pd.to_datetime(train_df['date'])
#train_df['week_day']=train_df['date'].dt.weekday_name
#train_df['month']=train_df['date'].dt.month
#train_df['year']=train_df['date'].dt.year
train_df=train_df.set_index(['date'])
train_df.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


In [6]:
train=train_df.loc[train_df.index<pd.to_datetime('October 1, 2017')]
test=train_df.loc[train_df.index>=pd.to_datetime('October 1, 2017')]
train.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


In [7]:
item_group_train=train.groupby(['store','item'])
item_group_test=test.groupby(['store','item'])

In [8]:
test_df['date']=pd.to_datetime(test_df['date'])
test_df=test_df.set_index(['date'])
test_item_group=test_df.groupby(['store','item'])

In [9]:
def SMAPE (forecast, actual):
    """Returns the Symmetric Mean Absolute Percentage Error between two Series"""
    try:
        diff = abs(forecast - actual)
        avg = abs(forecast) + abs(actual)
        m=diff/avg
    except ZeroDivisionError:
        m=0
    else:
        return(round(sum(m)/len(forecast) * 100, 2))

# ARIMA

In [10]:
# stationary test function

from statsmodels.tsa.stattools import adfuller
def stationarity_test(timeseries, window = 12, cutoff = 0.05):
    '''Performing Dickey_Fuller Test and return p-value'''
    
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20 )
    
    pvalue = dftest[1]
    
    return pvalue

In [11]:
def arima_model(df, start, end):
    """To build and store arima models for each item to be called for the groupby object."""
    
    # Step 1: stationary test to determine d
    #if stationarity_test(df.sales)<0.01:
        #d=0
    #else:
        #diff = df.sales - df.sales.shift(1)
        #diff = diff.dropna()
        #d=1
        #while stationarity_test(diff, window = 12)>0.01:
            #diff = diff - diff.shift(1)
            #diff = diff.dropna()
            #d=d+1
    
    # Step 2: Selection of p and q
    #resDiff = sm.tsa.arma_order_select_ic(diff, max_ar=7, max_ma=7, ic='aic', trend='c')
    #p=resDiff['aic_min_order'][0]
    #q=resDiff['aic_min_order'][1]
    
    # Step 3: Build arima model:
    arimax = sm.tsa.statespace.SARIMAX(df.sales, \
            order=(6,1,7),seasonal_order=(0,0,0,0),freq='D',\
            enforce_stationarity=False, enforce_invertibility=False).fit()
            #initialization='approximate_diffuse'
    
    # Step 4: Prediction
    
    return pd.DataFrame({'store':int(df['store'][0]),'item':int(df['item'][0]),'sales':arimax.predict(start,end,dynamic=True)})

In [12]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results=pd.DataFrame(columns=['store','item','sales'])


In [13]:

out = pd.concat(pool.starmap(arima_model, [(group,\
                                            list(item_group_test.get_group(name).index)[0],\
                                            list(item_group_test.get_group(name).index)[-1])\
                                            for name,group in item_group_train]))
pool.close()

In [14]:
out.head()

,store,item,sales
2017-10-01,1,1,27.959092
2017-10-02,1,1,19.523640
2017-10-03,1,1,21.937432
2017-10-04,1,1,23.451779
2017-10-05,1,1,23.915985


In [15]:
SMAPE(out['sales'],test['sales'])

27.36

# Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
# data preparation

df_pp=train_df.reset_index()
df_pp.columns=['ds','store','item','y']
df_pp

In [ ]:
train=df_pp.loc[df_pp['ds']<pd.to_datetime('October 1, 2017')]
test=df_pp.loc[df_pp['ds']>=pd.to_datetime('October 1, 2017')]

In [ ]:
# create groupby object
item_group_train=train.groupby(['store','item'])
item_group_test=test.groupby(['store','item'])

In [ ]:
def prophet_model(df_train,df_test):
    p=Prophet(interval_width=0.95, \
                  growth='linear',\
                  yearly_seasonality=False, \
                  weekly_seasonality=False, \
                  daily_seasonality=False,\
                  seasonality_mode='additive',\
                  seasonality_prior_scale=10,\
                  holidays_prior_scale=10,\
                  changepoint_prior_scale=10)
    p.add_country_holidays(country_name='US')
    p.add_seasonality(name='yearly', period=365.5, fourier_order=10, prior_scale=40)
    p.add_seasonality(name='weekly', period=7, fourier_order=3, prior_scale=10)
    p.add_seasonality(name='monthly', period=30.5, fourier_order=5, prior_scale=1)
    p.fit(df_train)
    return pd.DataFrame({'store':int(df_test['store'][0]),\
                         'item':int(df_test['item'][0]),\
                         'sales':p.predict(pd.DataFrame(df_test['ds']))['yhat']})

In [ ]:
pool = mp.Pool(mp.cpu_count())
out_prophet = pd.concat(pool.starmap(prophet_model, [(group[['ds','y']],\
                                                      item_group_test.get_group(name).reset_index())\
                                                      for name, group in item_group_train
                                                      ]))
pool.close()
out_prophet

In [ ]:
SMAPE(out_prophet.reset_index()['sales'],test.reset_index()['y'])